<a href="https://colab.research.google.com/github/pea-sys/Kaggle_Public/blob/master/ieee_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ieee-fraud-detectionコンペティションノートブック
クレジットカード不正利用を検知するモデルの精度を競う。

##結果
メダル：ソロ銀メダル  
順位：171/6381  
※上位5%に入ると銀メダル

##筆者の経験  
・機械学習を2018年12月から勉強開始  
・Pythonを2018年12月から勉強開始  
・CourseraのMachineLearning, DeepLearningの修了証取得済み  
・DeepoLearning G検定取得済み  
・その他書籍数冊  
・KaggleはtitanicとPorto seguroコンペに1サブミットのみ  
・本職ではない  
・開催中コンペの参加は初

##参加目的
・オンライン講座や本で勉強していたが、実践が不足していると感じたため。  
・Kaggleにしかない知見があるとのことで、興味があったため。

##感想  
・コンペティションを通じて成長できて良い経験になった。  
・順位を下げさせるスパム的なカーネルもあるので注意が必要（コメント要チェック）  
・pickleをロードして特徴量見ればモデルの再現は可能だけど、ソースをそのまま実行しても再現しないので、今後気を付けようと思う。  というかソースがゴミ箱みたいになっている。  
・アンサンブルの結果をLBで評価していたが、もっと良い評価方法を見つけたい。  
・GoogleColabは無料なので、あまり文句は言えないが計算中にセッション再接続→
インスタンス枯渇のコンボが毎日何回も出て進捗にかなり影響した。稼ぎがよわよわなので厳しい。  
・提出ファイルは２つ選べるので、自信が持てないなら必ず方向性の違う提出ファイルを2本用意した方が良い。  
・終始過学習していた。そのようなKernelも多かったように思う。training精度とvalidation精度は近づくようにするべきと講座では学んだが、どうするのが正解か分からなかった。inbalancedの場合はしょうがないのか？


##タイムライン
####7月序盤ー7月中盤  
　　開発環境構築。技術調査等。  
　　・[GoogleColabでの開発環境作成](https://github.com/pea-sys/Kaggle_Public/blob/master/KaggleTemplete.ipynb)  
　　・[Numba](https://github.com/pea-sys/Til/blob/master/NumbaPractice.ipynb)  
  　　・[Encode](https://github.com/pea-sys/Til/blob/master/CategoricalVariablesEncoding.ipynb)  
　　・[ImbalancedLearn](https://github.com/pea-sys/Til/blob/master/ImbalancedLearnPractice.ipynb)  
　　・[特徴量自動生成](https://github.com/pea-sys/Til/blob/master/AutoFeatureEngineering.ipynb)


####7月下旬  
　　・ KaggleのEDA公開カーネルやDisucussionを元にテーブルの匿名化されたカラムの感覚を掴む  
 　　・Googleスプレッドシートでの進捗管理開始  
 　　・取り合えず1submit  
　　・pickle化やdataframeオブジェクトのメモリ削除導入  
　　・ベースモデルをLGBMに決定  
　　・バリデーションの切り方を模索。この段階では教科書的にTimeSplitに。
  
####8月上旬  
　　・SMOTEを試すが精度上がらず。時間が特徴にあるので、  
　　　データ発生頻度が崩れてしまうと思い、サンプリング自体を断念。  
　　・アンサンブル用にXGBMもいじり出す  
　　・カーネルやディスカッションの特徴量を取り入れる  
　　・パラメータサーチ(無意味でした）
  
####8月中旬-下旬  
　　・場所毎のタイムデルタを特徴量に追加  
　　　(提供されているデータのタイムデルタが現地時間ではなくサーバの時間っぽかったので)  
　　・LGBMのgossやdartの勉強  
　　・特徴量選択(1個1個dropしたり、相関度を見ながら)  
　　・公開カーネルにPBで抜かれ始めて、焦ってアンサンブルを少しずつ始める  
　　・欠損値の処理方法検討（結局何も実践せず）  
　　・Kernelを参考に交互作用の特徴量を追加(FactorizeMachineは開発環境的な意味で断念)

####9月上旬
　　・バリデーションを月毎のGroupKFoldに変更  
　　(訓練データが数か月分で一年に満たないので、TimeSplitだと時系列データの序盤の予測精度が確認できない。また、実際評価は月毎にやると予想したため、バリデーションも評価単位に合わせるべきだと考えた。)


####9月中旬
　　・LGBMのfeatureImportanceを見て高ランクの特徴のタイムデルタを取ったものをひたすら作成した(効いてなさそうなものはdrop)  
    ・外部データを利用した特徴追加は精度上がらず（購買活動や犯罪率に影響しそうな温度や男女比などを国別に追加した。Amazonサイバーマンデー期間なども追加しようとしたけど辞めた）
  

####9月下旬-10月上旬
　　・かなり多忙だったため。機械的に毎日公開カーネルと自分のモデルをアンサンブルして5submitしていた（結果にはかなり貢献した）。






In [0]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

In [0]:
def gdrive_dl(filename):
  upload_filename = filename

  file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

  # ファイル ID を取得します。
  file_id = None
  for file in file_list:
    if file.get('name') == upload_filename:
      file_id = file.get('id')
      break

  if file_id is None:
    # ファイル ID を取得できなかった場合はエラーメッセージを出力します。
    print(upload_filename + ' が見つかりません.')
  else:
    # colab 環境へファイルをアップロードします。
    with open(upload_filename, 'wb') as f:
      request = drive_service.files().get_media(fileId=file_id)
      media = googleapiclient.http.MediaIoBaseDownload(f, request)

      done = False
      while not done:
        progress_status, done = media.next_chunk()
        print(100*progress_status.progress(), end="")
        print("%完了")

    print('GoogleドライブからColab環境へのファイル取り込みが完了しました.')

def gdrive_up(filename):

  file_metadata = {'name': filename,'mimeType': 'application/octet-stream'}
  media = googleapiclient.http.MediaFileUpload(filename,mimetype='application/octet-stream',resumable=True)
  created = drive_service.files().create(body=file_metadata,media_body=media, fields='id').execute()


In [0]:
gdrive_dl("kaggle.json")
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


In [0]:
!pip install kaggle
!kaggle competitions list --group 'entered'

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      10955            True  
competitive-data-science-predict-future-sales  2019-12-31 23:59:00  Playground           Kudos       4261            True  
severstal-steel-defect-detection               2019-10-24 23:59:00  Featured          $120,000       1865            True  
ieee-fraud-detection                           2019-10-03 23:59:00  Research           $20,000       6374            True  


In [0]:
gdrive_dl("sample_submission.pickle")
#gdrive_dl("train.pickle")
#gdrive_dl("test.pickle")
#gdrive_dl("train_df.pkl")
#gdrive_dl("test_df.pkl")
gdrive_dl("train_custom0917.pickle")
gdrive_dl("test_custom0917.pickle")

#gdrive_dl("train_transaction.csv.zip")
#gdrive_dl("test_transaction.csv.zip")

100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.
6.426338218382639%完了
12.852676436765279%完了
19.27901465514792%完了
25.705352873530558%完了
32.1316910919132%完了
38.55802931029584%完了
44.984367528678476%完了
51.410705747061115%完了
57.837043965443755%完了
64.2633821838264%完了
70.68972040220903%完了
77.11605862059167%完了
83.54239683897433%完了
89.96873505735695%完了
96.39507327573959%完了
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.
7.429130508077865%完了
14.85826101615573%完了
22.287391524233595%完了
29.71652203231146%完了
37.14565254038932%完了
44.57478304846719%完了
52.00391355654506%完了
59.43304406462292%完了
66.86217457270078%完了
74.29130508077864%完了
81.7204355888565%完了
89.14956609693438%完了
96.57869660501224%完了
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


### Library

In [0]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import os
import gc
import pickle
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline
pd.set_option('display.max_columns', 999)

### アンサンブルブロックSTART

In [0]:
"""
#Stacking 
sub = pd.read_pickle('sample_submission.pickle')
print(sub.shape)
gdrive_dl('FIX1003.csv')
gdrive_dl('9531FibonacciBlend.csv')


sub1 = pd.read_csv("FIX1003.csv")
sub2 = pd.read_csv("9531FibonacciBlend.csv")

sub['isFraud']=sub1['isFraud']*0.6+sub2['isFraud']*0.4
sub = sub.set_index('TransactionID')
sub.to_csv("FIX1003.csv")

gdrive_up('FIX1003.csv')
"""

(506691, 2)
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


In [0]:
#アンサンブル時のみコメントアウト
#!kaggle competitions submit -c ieee-fraud-detection -f FIX1003.csv -m FIX1003.csv

100% 14.1M/14.1M [00:00<00:00, 28.8MB/s]
Successfully submitted to IEEE-CIS Fraud Detection

### ヘルパー関数

In [0]:
import requests

def send_line_notification(message):
    line_token = 'line_token'  # 終わったら無効化する
    endpoint = 'https://notify-api.line.me/api/notify'
    message = "\n{}".format(message)
    payload = {'message': message}
    headers = {'Authorization': 'Bearer {}'.format(line_token)}
    requests.post(endpoint, data=payload, headers=headers)

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            dt_df[col] = dt_df[col].astype(float)  

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)
            dt_df['temp_mean'] = dt_df[period].map(temp_mean)
            dt_df['temp_std'] = dt_df[period].map(temp_std)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])
            dt_df[new_col+'_std_score'] = (dt_df[col]-dt_df['temp_mean'])/(dt_df['temp_std'])
            del dt_df['temp_min'],dt_df['temp_max'],dt_df['temp_mean'],dt_df['temp_std']
    return dt_df

In [0]:
def describe_add_isnull(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * mis_val / len(df)
  mis_val_table = pd.DataFrame(mis_val_percent)
  mis_val_table_ren_columns = mis_val_table.rename(columns={0:"Missing % of Total Values"})
  return pd.concat([df.describe(include='all'),mis_val_table_ren_columns.T])

In [0]:
def frequency_encoding(train, test, columns, self_encoding=False):
    for col in columns:
        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()
        if self_encoding:
            train[col] = train[col].map(fq_encode)
            test[col]  = test[col].map(fq_encode)            
        else:
            train[col+'_fq_enc'] = train[col].map(fq_encode)
            test[col+'_fq_enc']  = test[col].map(fq_encode)
    return train, test

In [0]:
def timeblock_frequency_encoding(train_df, test_df, periods, columns, 
                                 with_proportions=True, only_proportions=False):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            train_df[new_col] = train_df[col].astype(str)+'_'+train_df[period].astype(str)
            test_df[new_col]  = test_df[col].astype(str)+'_'+test_df[period].astype(str)

            temp_df = pd.concat([train_df[[new_col]], test_df[[new_col]]])
            fq_encode = temp_df[new_col].value_counts().to_dict()

            train_df[new_col] = train_df[new_col].map(fq_encode)
            test_df[new_col]  = test_df[new_col].map(fq_encode)
            
            if only_proportions:
                train_df[new_col] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col]  = test_df[new_col]/test_df[period+'_total']

            if with_proportions:
                train_df[new_col+'_proportions'] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col+'_proportions']  = test_df[new_col]/test_df[period+'_total']

    return train_df, test_df

## ADD START

In [0]:
def id_split(dataframe):
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1    
    return reduce_mem_usage(dataframe)

In [0]:
def add_card_dt(dataframe):

  card_dt = pd.DataFrame(dataframe['TransactionDT'] -  dataframe['TransactionDT'][0],columns=['card_dt'])
  visa_dt = dataframe['TransactionDT'][0] - 130
  mast_dt = dataframe['TransactionDT'][0] - 130
  amex_dt = dataframe['TransactionDT'][0] - 130
  disc_dt = dataframe['TransactionDT'][0] - 100

  for index, row in dataframe.iterrows():
    if row['card4'] == "visa":
      card_dt = card_dt.append(pd.Series([row['TransactionDT'] - visa_dt], index=card_dt.columns), ignore_index=True)
      visa_dt = row['TransactionDT']
    elif row['card4'] == "mastercard":
      card_dt = card_dt.append(pd.Series([row['TransactionDT'] - mast_dt], index=card_dt.columns), ignore_index=True)
      mast_dt = row['TransactionDT']
    elif row['card4'] == "american express":
      card_dt = card_dt.append(pd.Series([row['TransactionDT'] - amex_dt], index=card_dt.columns), ignore_index=True)
      amex_dt = row['TransactionDT']
    elif row['card4'] == "discover":
      card_dt = card_dt.append(pd.Series([row['TransactionDT'] - disc_dt], index=card_dt.columns), ignore_index=True)
      disc_dt = row['TransactionDT']
    else:
      card_dt = card_dt.append(pd.Series([np.nan], index=card_dt.columns), ignore_index=True)
    
  card_dt  = card_dt.fillna((dataframe['TransactionDT'][len(dataframe)-1] - dataframe['TransactionDT'][0])/card_dt['card_dt'].isnull().sum())
  dataframe = pd.concat([dataframe,card_dt], axis=1)
  del card_dt
  return reduce_mem_usage(dataframe) 

def add_card6_dt(dataframe):

  card6_dt = pd.DataFrame(dataframe['TransactionDT'] -  dataframe['TransactionDT'][0],columns=['card6_dt'])
  debit_dt = dataframe['TransactionDT'][0] - 50
  credit_dt = dataframe['TransactionDT'][0] - 50

  for index, row in dataframe.iterrows():
    if row['card6'] == 1:
      card6_dt = card6_dt.append(pd.Series([row['TransactionDT'] - debit_dt], index=card6_dt.columns), ignore_index=True)
      debit_dt = row['TransactionDT']
    elif row['card6'] == 2:
      card6_dt = card6_dt.append(pd.Series([row['TransactionDT'] - credit_dt], index=card6_dt.columns), ignore_index=True)
      credit_dt = row['TransactionDT']
    else:
      card6_dt = card6_dt.append(pd.Series([np.nan], index=card6_dt.columns), ignore_index=True)
    
  card6_dt  = card6_dt.fillna((dataframe['TransactionDT'][len(dataframe)-1] - dataframe['TransactionDT'][0])/card6_dt['card6_dt'].isnull().sum())
  dataframe = pd.concat([dataframe,card6_dt], axis=1)
  del card6_dt
  return reduce_mem_usage(dataframe) 

In [0]:
#タイムデルタの特徴作成(都度変更)
def add_M10_dt(dataframe):
  key = 'M10'
  addr1_dt = pd.DataFrame(dataframe['TransactionDT'] -  dataframe['TransactionDT'][0],columns=[key +'_dt'])
  dic = {}
  for index, row in dataframe.iterrows():
    if (row[key] in dic):
      addr1_dt = addr1_dt.append(pd.Series([row['TransactionDT'] - dic[row[key]]], index=addr1_dt.columns), ignore_index=True)
      dic[row[key]] = row['TransactionDT']
    else:
      dic.setdefault(row[key], row['TransactionDT'])
      addr1_dt = addr1_dt.append(pd.Series([np.nan], index=addr1_dt.columns), ignore_index=True)
    
  addr1_dt  = addr1_dt.fillna((dataframe['TransactionDT'][len(dataframe)-1] - dataframe['TransactionDT'][0])/addr1_dt[key + '_dt'].isnull().sum())
  dataframe = pd.concat([dataframe,addr1_dt], axis=1)
  del addr1_dt
  return reduce_mem_usage(dataframe) 

In [0]:
def add_missingCount_diff(df):
    premiss = pd.DataFrame(df['Missing_count'])
    premisssub = pd.DataFrame(df['Missing_count'][1:])
    premisssub.append(pd.Series( df['Missing_count'].mean()), ignore_index=True )
    premiss = premiss.reset_index() - premisssub.reset_index()
    premiss = pd.DataFrame(premiss['Missing_count'],columns=['Missing_count'])
    premiss.columns = ['Missing_count_diffAbs']
    premiss = premiss.abs()
    df = pd.concat([df,premiss], axis=1)
    return reduce_mem_usage(df)

In [0]:
 ### TransactionAmt mean addr1
def add_Amt_per_Local(df):
  adddf = df.groupby('addr2').mean()['TransactionAmt'].reset_index()
  adddf = adddf.rename(columns={'TransactionAmt': 'TransactionAmt_mean_addr2'})
  return pd.merge(df, adddf,  on='addr2',how = 'left')
 

In [0]:
def uid_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
    return train_df, test_df

def uid_aggregation_and_normalization(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            
            new_norm_col_name = col+'_'+main_column+'_std_norm'
            norm_cols = []
            
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
                norm_cols.append(new_col_name)
            
            train_df[new_norm_col_name] = (train_df[main_column]-train_df[norm_cols[0]])/train_df[norm_cols[1]]
            test_df[new_norm_col_name]  = (test_df[main_column]-test_df[norm_cols[0]])/test_df[norm_cols[1]]          
            
            del train_df[norm_cols[0]], train_df[norm_cols[1]]
            del test_df[norm_cols[0]], test_df[norm_cols[1]]
                                              
    return train_df, test_df

In [0]:
%%time
if not os.path.isfile('train_custom0917.pickle'):        
  train.to_pickle('train_custom.pickle')
  gdrive_up('train_custom.pickle')
else:
  train = pd.read_pickle('train_custom0917.pickle') 
  print('train=',train.shape)
  
if not os.path.isfile('test_custom0917.pickle'):
  test.to_pickle('test_custom.pickle')
  gdrive_up('test_custom.pickle')
else:
  test = pd.read_pickle('test_custom0917.pickle')
  print('test=',test.shape)  

if not os.path.isfile('sample_submission.pickle'):
  sub = reduce_mem_usage(pd.read_csv('sample_submission.csv.zip'))
  sub.to_pickle('sample_submission.pickle')
  gdrive_up('sample_submission.pickle')
else:
  sub = pd.read_pickle('sample_submission.pickle')
  print('sub=',sub.shape)

train= (590540, 926)
test= (506691, 925)
sub= (506691, 2)
CPU times: user 1.43 s, sys: 2.19 s, total: 3.62 s
Wall time: 3.65 s


In [0]:
#メモリの都合上。特徴追加後は一旦pickle化
train =  add_M10_dt(train)
test =  add_M10_dt(test)
train.to_pickle('train_custom0924.pickle')
gdrive_up('train_custom0924.pickle')
test.to_pickle('test_custom0924.pickle')
gdrive_up('test_custom0924.pickle')

In [0]:
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id,Missing_count,card_dt,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_std_addr1,addr1_dt,mail_dt,R_mail_dt,Missing_count_diff,Missing_count_diffAbs,TransactionAmt_decimal,card1_count_full,Transaction_day_of_week,Transaction_hour,id_02__id_20,id_02__D8,D11__DeviceInfo,DeviceInfo__P_emaildomain,P_emaildomain__C2,card2__dist1,card1__card5,card2__id_20,card5__P_emaildomain,addr1__card1,id_36_count_full,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist,ProductCD_target_mean,M4_target_mean,local_hour,local_hour_dist,M_sum,M_na,M_type,C_sum,C_null,C1_valid,C2_valid,C3_valid,C4_valid,C5_valid,C6_valid,C7_valid,C8_valid,C9_valid,C10_valid,C11_valid,C12_valid,C13_valid,C14_valid,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,C1_fq_enc,C2_fq_enc,C3_fq_enc,C4_fq_enc,C5_fq_enc,C6_fq_enc,C7_fq_enc,C8_fq_enc,C9_fq_enc,C10_fq_enc,C11_fq_enc,C12_fq_enc,C13_fq_enc,C14_fq_enc,D1_fq_enc,D2_fq_enc,D3_fq_enc,D4_fq_enc,D5_fq_enc,D6_fq_enc,D7_fq_enc,D8_fq_enc,D9_fq_enc,addr1_fq_enc,addr2_fq_enc,dist1_fq_enc,dist2_fq_enc,P_emaildomain_fq_enc,R_emaildomain_fq_enc,id_01_fq_enc,id_02_fq_enc,id_03_fq_enc,id_04_fq_enc,id_05_fq_enc,id_06_fq_enc,id_07_fq_enc,id_08_fq_enc,id_09_fq_enc,id_10_fq_enc,id_11_fq_enc,id_13_fq_enc,id_14_fq_enc,id_17_fq_enc,id_18_fq_enc,id_19_fq_enc,id_20_fq_enc,id_21_fq_enc,id_22_fq_enc,id_24_fq_enc,id_25_fq_enc,id_26_fq_enc,id_30_fq_enc,id_31_fq_enc,id_32_fq_enc,id_33_fq_enc,DeviceInfo_fq_enc,uid,uid2,Transac

In [0]:
#0903
columns_a = ['TransactionAmt','D15']#'id_02', 
columns_b = ['addr1','card1','card4']  

for col_a in columns_a:
    for col_b in columns_b:
        for df in [train, test]:
            df[f'{col_a}_to_mean_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('mean')
            df[f'{col_a}_to_std_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('std')

In [0]:
'''
#温度 addr2
#87 US
#16 65 Rossia
#32 Canada
#60,96 Europe
#http://www.data.jma.go.jp/gmd/cpd/monitor/climatview/graph_mkhtml.php?&n=72295&p=12&s=7&r=0&y=2017&m=9&e=0&k=0&d=2

for df in [train, test]:
  df['temp_av'] = np.nan
  df['temp_max'] = np.nan
  df['temp_min'] = np.nan
  df['precipitation'] = np.nan
  df['temp_std'] = np.nan
  df['precipitation_std'] = np.nan
  
#2017年11月スタート
for df in [train, test]:
  ## US　	NY				
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'temp_av'] = 8.9
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'temp_max'] = 12.8
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'temp_min'] = 5.0
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'precipitation'] = 52	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'temp_std'] = 9.2
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 2),'precipitation_std'] = 87.8
 	#2017.12				
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'temp_av'] = 2.4	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'temp_max'] = 5.1	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'temp_min'] = -0.3
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'precipitation'] =52
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'temp_std'] = 3.7
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 3),'precipitation_std'] = 90.3
	#2018.1					
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'temp_av'] = -0.3	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'temp_max'] =3.5	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'temp_min'] = -4.1
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'precipitation'] = 54
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'temp_std'] = 1
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 4),'precipitation_std'] = 82.5	
	#2018.2										
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'temp_av'] = 5.8	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'temp_max'] =10.0	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'temp_min'] = 1.6
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'precipitation'] =151
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'temp_std'] = 2.0
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 5),'precipitation_std'] = 67.8
  #2018.3 							
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'temp_av'] = 4.9	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'temp_max'] = 8.1
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'temp_min'] = 1.7
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'precipitation'] =104
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'temp_std'] = 5.9
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 6),'precipitation_std'] = 105.1
  #2018.4 9.7	13.6	5.7	128	11.6	102.1	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'temp_av'] = 9.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'temp_max'] = 13.6	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'temp_min'] = 5.7
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'precipitation'] =128
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'temp_std'] = 11.6
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 7),'precipitation_std'] = 102.1
  #2018.5 19.7	24.6	14.7	82	17.1	97.3
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'temp_av'] = 19.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'temp_max'] = 24.6	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'temp_min'] = 14.7
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'precipitation'] =82
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'temp_std'] = 17.1
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 8),'precipitation_std'] = 97.3
  #2018.6 22.7	27.1	18.4	92	22.4	101.8
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'temp_av'] = 22.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'temp_max'] = 27.1	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'temp_min'] = 18.4
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'precipitation'] =92
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'temp_std'] = 22.4
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 9),'precipitation_std'] = 101.8
  #2018,7 26.6	30.6	22.6	134	25.3	111.4
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'temp_av'] = 26.6	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'temp_max'] = 30.6	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'temp_min'] = 22.6
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'precipitation'] =134
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'temp_std'] = 25.3
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 10),'precipitation_std'] = 111.4
	#2018.8 27.2	30.9	23.5	169	24.8	107.9	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'temp_av'] = 27.2	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'temp_max'] = 30.9	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'temp_min'] = 23.5
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'precipitation'] =169
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'temp_std'] = 24.8
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 11),'precipitation_std'] = 107.9
  #2018.9 22.7	25.7	19.6	150	20.8	94.5
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'temp_av'] = 22.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'temp_max'] = 25.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'temp_min'] = 19.6
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'precipitation'] = 150
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'temp_std'] = 20.8
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 0),'precipitation_std'] = 94.5
  #2018.10  15.5	18.7	12.2	77	14.7	96.9
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'temp_av'] = 15.5	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'temp_max'] = 18.7	
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'temp_min'] = 12.2
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'precipitation'] = 77
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'temp_std'] = 14.7
  df.loc[(df['addr2'] == 87) & (df['DT_M'] == 1),'precipitation_std'] = 96.9
  
  #16 65 Rossia Mosquwa
  #2017.11 0.0	1.6	-1.6	46	-1.2	54.9
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'temp_av'] = 0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'temp_max'] = 1.6
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'temp_min'] = -1.6
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'precipitation'] = 46	
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'temp_std'] = -1.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 2),'precipitation_std'] = 54.9
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'temp_av'] = 0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'temp_max'] = 1.6
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'temp_min'] = -1.6
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'precipitation'] = 46	
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'temp_std'] = -1.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 2),'precipitation_std'] = 54.9
  #2017.12 0.0	1.4	-1.5	86	-5.2	50.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'temp_av'] = 0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'temp_max'] = 1.4
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'temp_min'] = -1.5
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'precipitation'] = 86	
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'temp_std'] = -5.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 3),'precipitation_std'] = 50.3
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'temp_av'] = 0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'temp_max'] = 1.4
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'temp_min'] = -1.5
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'precipitation'] = 86	
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'temp_std'] = -5.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 3),'precipitation_std'] = 50.3
  #2018.1 -4.3	-2.3	-6.2	66	-6.5	51.6
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'temp_av'] = -4.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'temp_max'] = -2.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'temp_min'] = -6.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'precipitation'] = 66
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'temp_std'] = -6.5
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 4),'precipitation_std'] = 51.6
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'temp_av'] = -4.3
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'temp_max'] = -2.3
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'temp_min'] = -6.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'precipitation'] = 66
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'temp_std'] = -6.5
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 4),'precipitation_std'] = 51.6
  
  #2018.2 -9.1	-6.2	-12.1	64	-6.7	43.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'temp_av'] = -9.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'temp_max'] = -6.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'temp_min'] = -12.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'precipitation'] = 64
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'temp_std'] = -6.7
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 5),'precipitation_std'] = 43.1
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'temp_av'] = -9.1
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'temp_max'] = -6.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'temp_min'] = -12.1
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'precipitation'] = 64
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'temp_std'] = -6.7
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 5),'precipitation_std'] = 43.1
  
  #2018.3 -5.1	-1.0	-9.2	31	-1.0	35.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'temp_av'] = -5.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'temp_max'] = -1.0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'temp_min'] = -9.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'precipitation'] = 31
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'temp_std'] = -1.0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 6),'precipitation_std'] = 35.2
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'temp_av'] = -5.1
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'temp_max'] = -1.0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'temp_min'] = -9.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'precipitation'] = 31
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'temp_std'] = -1.0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 6),'precipitation_std'] = 35.2
  
  #2018.4 7.8	13.5	2.5	47	6.7	36.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'temp_av'] = 7.8
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'temp_max'] = 13.5
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'temp_min'] = 2.5
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'precipitation'] = 47
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'temp_std'] = 6.7
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 7),'precipitation_std'] = 36.3
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'temp_av'] = 7.8
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'temp_max'] = 13.5
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'temp_min'] = 2.5
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'precipitation'] = 47
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'temp_std'] = 6.7
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 7),'precipitation_std'] = 36.3
  
  #2018.5 16.2	22.1	10.2	61	13.2	50.3 
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'temp_av'] = 16.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'temp_max'] = 22.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'temp_min'] = 10.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'precipitation'] = 61
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'temp_std'] = 13.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 8),'precipitation_std'] = 50.3
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'temp_av'] = 16.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'temp_max'] = 22.1
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'temp_min'] = 10.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'precipitation'] = 61
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'temp_std'] = 13.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 8),'precipitation_std'] = 50.3
  
  #2018.6 17.3	22.9	11.7	56	17.0	80.4	
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'temp_av'] = 17.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'temp_max'] = 22.9
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'temp_min'] = 11.7
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'precipitation'] = 56
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'temp_std'] = 17
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 9),'precipitation_std'] =80.4
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'temp_av'] = 17.3
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'temp_max'] = 22.9
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'temp_min'] = 11.7
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'precipitation'] = 56
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'temp_std'] = 17
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 9),'precipitation_std'] =80.4
   
  #2018.7 20.5	24.9	16.7	92	19.2	84.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'temp_av'] = 20.5
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'temp_max'] = 24.9
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'temp_min'] = 16.7
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'precipitation'] = 92
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'temp_std'] = 19.2
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 10),'precipitation_std'] =84.3
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'temp_av'] = 20.5
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'temp_max'] = 24.9
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'temp_min'] = 16.7
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'precipitation'] = 92
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'temp_std'] = 19.2
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 10),'precipitation_std'] =84.3
  
  #2018.8 19.8	25.8	13.7	28	17.0	82.0	
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'temp_av'] = 19.8
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'temp_max'] = 25.8
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'temp_min'] = 13.7
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'precipitation'] = 28
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'temp_std'] = 17
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 11),'precipitation_std'] =82.0
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'temp_av'] = 19.8
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'temp_max'] = 25.8
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'temp_min'] = 13.7
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'precipitation'] = 28
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'temp_std'] = 17
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 11),'precipitation_std'] =82.0
  
  #2018.9 14.6	20.0	10.0	77	11.3	66.8
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'temp_av'] = 14.6
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'temp_max'] = 20.0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'temp_min'] = 10.0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'precipitation'] = 77
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'temp_std'] = 11.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 0),'precipitation_std'] =66.8
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'temp_av'] = 14.6
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'temp_max'] = 20.0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'temp_min'] = 10.0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'precipitation'] = 77
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'temp_std'] = 11.3
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 0),'precipitation_std'] =66.8
  
  #2018.10 7.3	11.1	4.0	56	5.6	71.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'temp_av'] = 7.3
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'temp_max'] = 11.1
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'temp_min'] = 4.0
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'precipitation'] = 56
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'temp_std'] =5.6
  df.loc[(df['addr2'] == 16) & (df['DT_M'] == 1),'precipitation_std'] =71.3
  
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'temp_av'] = 7.3
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'temp_max'] = 11.1
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'temp_min'] = 4.0
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'precipitation'] = 56
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'temp_std'] =5.6
  df.loc[(df['addr2'] == 65) & (df['DT_M'] == 1),'precipitation_std'] =71.3
  
  #32 Canada 
  #2017.11 0.6	5.0	-3.9	86
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 2),'temp_av'] = 0.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 2),'temp_max'] = 5.0
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 2),'temp_min'] = -3.9
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 2),'precipitation'] = 86
  
  #2017.12 -8.8	-5.5	-11.0	24	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 3),'temp_av'] = -8.8
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 3),'temp_max'] = -5.5
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 3),'temp_min'] = -11
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 3),'precipitation'] = 24
  
  #2018.1 -10.2	-5.1	-15.2	67
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 4),'temp_av'] = -10.2
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 4),'temp_max'] = -5.1
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 4),'temp_min'] = -15.2
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 4),'precipitation'] = 67
  
  #2018.2 -5.3	0.1	-10.7	59
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 5),'temp_av'] = -5.3
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 5),'temp_max'] = -0.1
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 5),'temp_min'] = -10.7
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 5),'precipitation'] = 59
  
  #2018.3 -1.6	1.8	-4.8	45	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 6),'temp_av'] = -1.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 6),'temp_max'] = 1.8
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 6),'temp_min'] = -4.8
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 6),'precipitation'] = 45

  #2018.4 2.7	7.2	-1.7	81
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 7),'temp_av'] = 2.7
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 7),'temp_max'] = 7.2
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 7),'temp_min'] = -1.7
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 7),'precipitation'] = 81
  
  #2018.5 15.4	22.0	8.6	44
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 8),'temp_av'] = 15.4
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 8),'temp_max'] = 22
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 8),'temp_min'] = 8.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 8),'precipitation'] = 44
  
  #2018.6 18.2	24.1	12.3	65	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 9),'temp_av'] = 18.2
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 9),'temp_max'] = 24.1
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 9),'temp_min'] = 12.3
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 9),'precipitation'] = 65
  
  #2018.7 23.1	29.6	16.6	195	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 10),'temp_av'] = 23.1
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 10),'temp_max'] = 29.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 10),'temp_min'] = 16.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 10),'precipitation'] = 195
  
  #2018.8 21.5	26.7	16.4	69	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 11),'temp_av'] = 21.5
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 11),'temp_max'] = 26.7
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 11),'temp_min'] = 16.4
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 11),'precipitation'] = 69
  
  #2018.9 16.6	22.1	10.9	76	
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 0),'temp_av'] = 16.6
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 0),'temp_max'] = 22.1
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 0),'temp_min'] = 10.9
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 0),'precipitation'] = 76
  
  #2018.10 7.0	10.8	3.0	53
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 1),'temp_av'] = 7
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 1),'temp_max'] = 10.8
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 1),'temp_min'] = 3
  df.loc[(df['addr2'] == 32) & (df['DT_M'] == 1),'precipitation'] = 53
  
  #60,96 Europe
  #2017.11 5.8	8.3	3.1	69	5.4	
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 2),'temp_av'] = 5.8
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 2),'temp_max'] = 8.3
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 2),'temp_min'] = 3.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 2),'precipitation'] = 69
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 2),'temp_std'] =5.4
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 2),'temp_av'] = 5.8
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 2),'temp_max'] = 8.3
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 2),'temp_min'] = 3.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 2),'precipitation'] = 69
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 2),'temp_std'] =5.4
  
  #2017.12 3.5	5.7	1.2	40	1.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 3),'temp_av'] = 3.5
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 3),'temp_max'] = 5.7
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 3),'temp_min'] = 1.2
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 3),'precipitation'] = 40
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 3),'temp_std'] =1.6
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 3),'temp_av'] = 3.5
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 3),'temp_max'] = 5.7
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 3),'temp_min'] = 1.2
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 3),'precipitation'] = 40
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 3),'temp_std'] =1.6
  
  #2018.1  3.5	5.6	1.2	67	1.3
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 4),'temp_av'] = 3.5
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 4),'temp_max'] = 5.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 4),'temp_min'] = 1.2
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 4),'precipitation'] = 67
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 4),'temp_std'] =1.3
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 4),'temp_av'] = 3.5
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 4),'temp_max'] = 5.6
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 4),'temp_min'] = 1.2
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 4),'precipitation'] = 67
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 4),'temp_std'] =1.3
  
  #2018.2 -0.9	2.7	-4.1	3	2.0
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 5),'temp_av'] = 0.9
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 5),'temp_max'] = 2.7
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 5),'temp_min'] = -4.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 5),'precipitation'] = 3
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 5),'temp_std'] =2.0
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 5),'temp_av'] = 0.9
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 5),'temp_max'] = 2.7
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 5),'temp_min'] = -4.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 5),'precipitation'] = 3
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 5),'temp_std'] =2.0
  
  #2018.3 1.7	5.6	-2.1	42	4.8	
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_av'] = 1.7
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_max'] = 5.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_min'] = -2.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'precipitation'] = 42
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_std'] =4.8
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_av'] = 1.7
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_max'] = 5.6
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_min'] = -2.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'precipitation'] = 42
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_std'] =4.8
  
  #2018.4 13.4	19.5	7.1	41	10.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_av'] = 13.4
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_max'] = 19.5
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_min'] = 7.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'precipitation'] = 41
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 6),'temp_std'] =10.1
  
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_av'] = 13.4
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_max'] = 19.5
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_min'] = 7.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'precipitation'] = 41
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 6),'temp_std'] =10.1
  
  #2018.5 17.6	23.9	10.8	13	14.2
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 7),'temp_av'] = 17.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 7),'temp_max'] = 23.9
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 7),'temp_min'] = 10.8
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 7),'precipitation'] = 13
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 7),'temp_std'] =14.2
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 7),'temp_av'] = 17.6
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 7),'temp_max'] = 23.9
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 7),'temp_min'] = 10.8
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 7),'precipitation'] = 13
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 7),'temp_std'] =14.2
  
  #2018.6 19.0	24.9	13.8	15	17.8
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 8),'temp_av'] = 19.0
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 8),'temp_max'] = 24.9
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 8),'temp_min'] = 13.8
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 8),'precipitation'] = 15
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 8),'temp_std'] =17.8
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 8),'temp_av'] = 19.0
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 8),'temp_max'] = 24.9
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 8),'temp_min'] = 13.8
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 8),'precipitation'] = 15
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 8),'temp_std'] =17.8
  
  #2018.7 21.1	27.1	14.9	71	19.7	
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 9),'temp_av'] = 21.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 9),'temp_max'] = 27.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 9),'temp_min'] = 14.9
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 9),'precipitation'] = 71
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 9),'temp_std'] =19.7
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 9),'temp_av'] = 21.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 9),'temp_max'] = 27.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 9),'temp_min'] = 14.9
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 9),'precipitation'] = 71
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 9),'temp_std'] =19.7
  
  #2018.9 16.1	22.6	10.8	20	14.4	
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 11),'temp_av'] = 16.1
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 11),'temp_max'] = 22.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 11),'temp_min'] = 10.8
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 11),'precipitation'] = 20
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 11),'temp_std'] =14.4
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 11),'temp_av'] = 16.1
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 11),'temp_max'] = 22.6
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 11),'temp_min'] = 10.8
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 11),'precipitation'] = 20
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 11),'temp_std'] =14.4
  
  #2018.10 10.7	16.5	5.9	18	8.7	
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 0),'temp_av'] = 10.7
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 0),'temp_max'] = 16.5
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 0),'temp_min'] = 5.9
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 0),'precipitation'] = 18
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 0),'temp_std'] = 8.7
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 0),'temp_av'] = 10.7
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 0),'temp_max'] = 16.5
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 0),'temp_min'] = 5.9
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 0),'precipitation'] = 18
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 0),'temp_std'] = 8.7
  
  #2018.11 5.3	8.4	2.6	15	5.4
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 1),'temp_av'] = 5.3
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 1),'temp_max'] = 8.4
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 1),'temp_min'] = 2.6
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 1),'precipitation'] = 15
  df.loc[(df['addr2'] == 60) & (df['DT_M'] == 1),'temp_std'] = 5.4
  
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 1),'temp_av'] = 5.3
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 1),'temp_max'] = 8.4
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 1),'temp_min'] = 2.6
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 1),'precipitation'] = 15
  df.loc[(df['addr2'] == 96) & (df['DT_M'] == 1),'temp_std'] = 5.4
  '''

In [0]:
'''
#男女比 addr2
#87 US
#16 65 Rossia
#32 Canada
#60,96 Europe
#http://www.data.jma.go.jp/gmd/cpd/monitor/climatview/graph_mkhtml.php?&n=72295&p=12&s=7&r=0&y=2017&m=9&e=0&k=0&d=2

for df in [train, test]:
  df['gender'] = np.nan
  df.loc[(df['addr2'] == 87),'gender'] = 1.06
  #16 65 Rossia
  df.loc[(df['addr2'] == 16),'gender'] = 1.06
  df.loc[(df['addr2'] == 65),'gender'] = 1.06
  #32 Canada
  df.loc[(df['addr2'] == 32),'gender'] = 1.056
  #60,96 Europe
  df.loc[(df['addr2'] == 60),'gender'] = 1.058
  df.loc[(df['addr2'] == 96),'gender'] = 1.058
'''

In [0]:
# Add list of feature that we will
# remove later from final features list
remove_features = []

In [0]:
########################### TransactionDT
'''0902
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
dates_range = pd.date_range(start='2017-10-01', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
for df in [train, test]:
    # Temporary variables for aggregation
    df['DT'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
    df['DT_M'] = ((df['DT'].dt.year-2017)*12 + df['DT'].dt.month).astype(np.int8) % 12
    #6月を対称に取引データは増える（対称性を表現）
    df['DT_M_ABS'] = abs(df['DT_M'] - 6)
    df['DT_W'] = ((df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear).astype(np.int8)
    df['DT_D'] = ((df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear).astype(np.int16)
    
    df['DT_hour'] = (df['DT'].dt.hour).astype(np.int8)
    df['DT_day_week'] = (df['DT'].dt.dayofweek).astype(np.int8)
    df['DT_day_month'] = (df['DT'].dt.day).astype(np.int8)
        
    # Possible solo feature
    df['is_december'] = df['DT'].dt.month
    df['is_december'] = (df['is_december']==12).astype(np.int8)

    # Holidays
    df['is_holiday'] = (df['DT'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
    

# Remove temporary features from final list
remove_features += ['DT','DT_M','DT_W','DT_D','DT_hour','DT_day_week','DT_day_month']

# Total transactions per timeblock
for col in ['DT_M','DT_W','DT_D']:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
            
    train[col+'_total'] = train[col].map(fq_encode)
    test[col+'_total']  = test[col].map(fq_encode)
    
    # We can't use it as solo feature
    remove_features.append(col+'_total')
'''
remove_features += ['DT','DT_M','DT_W','DT_D','DT_hour','DT_day_week','DT_day_month']
#remove_features += ['DT_total','DT_M_total','DT_W_total','DT_D_total','DT_hour_total','DT_day_week_total','DT_day_month_total']

In [0]:
'''0902
#0831ADD
########################### Card columns "outliers"
for col in ['card1']: 
    valid_card = pd.concat([train[[col]], test[[col]]])
    valid_card = valid_card[col].value_counts()
    valid_card_std = valid_card.values.std()

    invalid_cards = valid_card[valid_card<=2]
    print('Rare cards',len(invalid_cards))

    valid_card = valid_card[valid_card>2]
    valid_card = list(valid_card.index)

    print('No intersection in Train', len(train[~train[col].isin(test[col])]))
    print('Intersection in Train', len(train[train[col].isin(test[col])]))
    
    train[col] = np.where(train[col].isin(test[col]), train[col], np.nan)
    test[col]  = np.where(test[col].isin(train[col]), test[col], np.nan)

    train[col] = np.where(train[col].isin(valid_card), train[col], np.nan)
    test[col]  = np.where(test[col].isin(valid_card), test[col], np.nan)
    print('#'*20)

for col in ['card2','card3','card4','card5','card6',]: 
    print('No intersection in Train', col, len(train[~train[col].isin(test[col])]))
    print('Intersection in Train', col, len(train[train[col].isin(test[col])]))
    
    train[col] = np.where(train[col].isin(test[col]), train[col], np.nan)
    test[col]  = np.where(test[col].isin(train[col]), test[col], np.nan)
    print('#'*20)
'''

In [0]:
print(train.shape,test.shape)
describe_add_isnull(train)

In [0]:
'''0902

#20190831 ADD
########################### bank_type
# Tracking nomal activity
# by doing timeblock frequency encoding
i_cols = ['bank_type'] #['uid','uid2','uid3','uid4','uid5','bank_type']
periods = ['DT_M','DT_W','DT_D']

# We have few options to encode it here:
# - Just count transactions
# (but some timblocks have more transactions than others)
# - Devide to total transactions per timeblock (proportions)
# - Use both
# - Use only proportions
train, test = timeblock_frequency_encoding(train, test, periods, i_cols, 
                                 with_proportions=False, only_proportions=True)
'''

In [0]:
'''0902
train["diff_V319_V320"] = np.zeros(train.shape[0])

train.loc[train["V319"]!=train["V320"],"diff_V319_V320"] = 1

test["diff_V319_V320"] = np.zeros(test.shape[0])

test.loc[test["V319"]!=test["V320"],"diff_V319_V320"] = 1

train["diff_V109_V110"] = np.zeros(train.shape[0])

train.loc[train["V109"]!=train["V110"],"diff_V109_V110"] = 1

test["diff_V109_V110"] = np.zeros(test.shape[0])

test.loc[test["V109"]!=test["V110"],"diff_V109_V110"] = 1

train["diff_V4_V5"] = np.zeros(train.shape[0])

train.loc[train["V4"]!=train["V5"],"diff_V4_V5"] = 1

test["diff_V4_V5"] = np.zeros(test.shape[0])

test.loc[test["V4"]!=test["V5"],"diff_V4_V5"] = 1
'''

#ここから下は未使用

##train["diff_V329_V330"] = np.zeros(train.shape[0])

##train.loc[train["V329"]!=train["V330"],"diff_V329_V330"] = 1

##test["diff_V329_V330"] = np.zeros(test.shape[0])

##test.loc[test["V329"]!=test["V330"],"diff_V329_V330"] = 1


##train["diff_V316_V331"] = np.zeros(train.shape[0])

##train.loc[train["V331"]!=train["V316"],"diff_V316_V331"] = 1

##test["diff_V316_V331"] = np.zeros(test.shape[0])

##test.loc[test["V331"]!=test["V316"],"diff_V316_V331"] = 1


In [0]:
'''0902
#20190825 ADD      
# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))

#20190831 ADD
train['card2_count_full'] = train['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))
test['card2_count_full'] = test['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))

'''

In [0]:
#09/05　カラム数かわっちゃう
"""
# Create Features based on anonymised prefix groups
prefix = ['C','D','Device','M','Transaction','V','addr','card','dist']#,'id'
for df in [train, test]:
  for i, p in enumerate(prefix):
    column_set = [x for x in df.columns.tolist() if x.startswith(prefix[i])]
    # Take NA count
    df[p + "group_nan_sum"] = df[column_set].isnull().sum(axis=1) / df[column_set].shape[1]

    # Take SUM/Mean if numeric
    numeric_cols = [x for x in column_set if df[x].dtype != object]
    if numeric_cols:
      df[p + "group_sum"] = df[column_set].sum(axis=1)
      df[p + "group_mean"] = df[column_set].mean(axis=1)
      # Zero Count
      df[p + "group_0_count"] = (df[column_set] == 0).astype(int).sum(axis=1) / (df[column_set].shape[1] - df[p + "group_nan_sum"])
"""

In [0]:
#20190828
'''
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
train["Date"] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
train['_Weekdays'] = train['Date'].dt.dayofweek
train['_Hours'] = train['Date'].dt.hour
train['_Days'] = train['Date'].dt.day

test["Date"] = test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
test['_Weekdays'] = test['Date'].dt.dayofweek
test['_Hours'] = test['Date'].dt.hour
test['_Days'] = test['Date'].dt.day

train = train.drop("Date",axis=1)
test = test.drop("Date",axis=1)

train['_Days'] = train['_Days'].astype("category")
train['_Weekdays'] = train['_Weekdays'].astype("category")
test['_Days'] = test['_Days'].astype("category")
test['_Weekdays'] = test['_Weekdays'].astype("category")


###########################  Target mean
for col in ['_Weekdays','_Days','_Hours']:
    temp_dict = train.groupby([col])['isFraud'].agg(['mean']).reset_index().rename(
                                                        columns={'mean': col+'_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col+'_target_mean'].to_dict()

    train[col+'_target_mean'] = train[col].map(temp_dict)
    test[col+'_target_mean']  = test[col].map(temp_dict)
'''

In [0]:
'''0902
#20190831
########################### 'P_emaildomain' - 'R_emaildomain'
# Matching
train['email_check'] = np.where(train['P_emaildomain']==train['R_emaildomain'],1,0)
test['email_check']  = np.where(test['P_emaildomain']==test['R_emaildomain'],1,0)

# All NaNs
train['email_check_nan_all'] = np.where((train['P_emaildomain'].isna())&(train['R_emaildomain'].isna()),1,0)
test['email_check_nan_all']  = np.where((test['P_emaildomain'].isna())&(test['R_emaildomain'].isna()),1,0)

# Any NaN
train['email_check_nan_any'] = np.where((train['P_emaildomain'].isna())|(train['R_emaildomain'].isna()),1,0)
test['email_check_nan_any']  = np.where((test['P_emaildomain'].isna())|(test['R_emaildomain'].isna()),1,0)

# Fix NaN, get "prefix"
def fix_emails(df):
    df['P_emaildomain'] = df['P_emaildomain'].fillna('email_not_provided')
    df['R_emaildomain'] = df['R_emaildomain'].fillna('email_not_provided')
    
    df['email_match_not_nan'] = np.where((df['P_emaildomain']==df['R_emaildomain'])&
                                     (df['P_emaildomain']!='email_not_provided'),1,0)
    
    #df['P_email_prefix'] = df['P_emaildomain'].apply(lambda x: x.split('.')[0])
    #df['R_email_prefix'] = df['R_emaildomain'].apply(lambda x: x.split('.')[0])
    return df

train = fix_emails(train)
test = fix_emails(test)
'''

In [0]:
'''
########################### D9 and TransactionDT
# Seems that D9 column is a hour
# But what hour?
# Local time? Server time? Shop time?
# Previous transaction? Most common time for client?
# Is there difference between TransactionDT and D9 column?
# Is there connection with distance?
train['local_hour'] = train['D9']*24
test['local_hour']  = test['D9']*24

train['local_hour'] = train['local_hour'] - (train['TransactionDT']/(60*60))%24
test['local_hour']  = test['local_hour'] - (test['TransactionDT']/(60*60))%24

train['local_hour_dist'] = train['local_hour']/train['dist2']
test['local_hour_dist']  = test['local_hour']/test['dist2']
'''

In [0]:
"""
########################### M columns (except M4)
# All these columns are binary encoded 1/0
# We can have some features from it
i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']

train['M_sum'] = train[i_cols].sum(axis=1).astype(np.int8)
test['M_sum']  = test[i_cols].sum(axis=1).astype(np.int8)

train['M_na'] = train[i_cols].isna().sum(axis=1).astype(np.int8)
test['M_na']  = test[i_cols].isna().sum(axis=1).astype(np.int8)

train['M_type'] = ''
test['M_type']  = ''

for col in i_cols:
    train['M_type'] = '_'+train[col].astype(str)
    test['M_type'] = '_'+test[col].astype(str)
"""

In [0]:
'''
# 20190901
########################### C Columns
i_cols = ['C'+str(i) for i in range(1,15)]

####### Global Self frequency encoding
# self_encoding=False because 
# I want to keep original values
train, test = frequency_encoding(train, test, i_cols, self_encoding=False)

####### Clip max values
for df in [train, test]:
    for col in i_cols:
        max_value = train[train['DT_M']==train['DT_M'].max()][col].max()
        df[col] = df[col].clip(None,max_value) 
'''

In [0]:
"""
########################### C columns
# C columns are some counts, based on client identity
# Most popular Value is "1" -> that seems to be just a single match 
# (New or stable client)
# You can check that auc score for that cliens are lower than global
# Lets encode such client types

i_cols = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14']

train['C_sum'] = 0
test['C_sum']  = 0

train['C_null'] = 0
test['C_null']  = 0

for col in i_cols:
    train['C_sum'] += np.where(train[col]==1,1,0)
    test['C_sum']  += np.where(test[col]==1,1,0)

    train['C_null'] += np.where(train[col]==0,1,0)
    test['C_null']  += np.where(test[col]==0,1,0)
    
    valid_values = train[col].value_counts()
    valid_values = valid_values[valid_values>1000]
    valid_values = list(valid_values.index)
    
    train[col+'_valid'] = np.where(train[col].isin(valid_values),1,0)
    test[col+'_valid']  = np.where(test[col].isin(valid_values),1,0)
"""

In [0]:
"""
########################### Reset values for "noise" card1
valid_card = train['card1'].value_counts()
valid_card = valid_card[valid_card>10]
valid_card = list(valid_card.index)
    
train['card1'] = np.where(train['card1'].isin(valid_card), train['card1'], np.nan)
test['card1']  = np.where(test['card1'].isin(valid_card), test['card1'], np.nan)
"""

In [0]:
'''0902
########################### Freq encoding
i_cols = ['card1','card2','card3','card5',
          'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
          'D1','D2','D3','D4','D5','D6','D7','D8','D9',
          'addr1','addr2',
          'dist1','dist2',
          'P_emaildomain', 'R_emaildomain',
          'id_01','id_02','id_03','id_04','id_05','id_06','id_07','id_08','id_09','id_10',
          'id_11','id_13','id_14','id_17','id_18','id_19','id_20','id_21','id_22','id_24',
          'id_25','id_26','id_30','id_31','id_32','id_33','DeviceInfo','uid','uid2']		

for col in i_cols:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()   
    train[col+'_fq_enc'] = train[col].map(fq_encode)
    test[col+'_fq_enc']  = test[col].map(fq_encode)
'''
   
"""未使用
  
for col in ['DT_M','DT_W','DT_D']:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
            
    train[col+'_total'] = train[col].map(fq_encode)
    test[col+'_total']  = test[col].map(fq_encode)
        

periods = ['DT_M','DT_W','DT_D']
i_cols = ['uid']
for period in periods:
    for col in i_cols:
        new_column = col + '_' + period
            
        temp_df = pd.concat([train[[col,period]], test[[col,period]]])
        temp_df[new_column] = temp_df[col].astype(str) + '_' + (temp_df[period]).astype(str)
        fq_encode = temp_df[new_column].value_counts().to_dict()
            
        train[new_column] = (train[col].astype(str) + '_' + train[period].astype(str)).map(fq_encode)
        test[new_column]  = (test[col].astype(str) + '_' + test[period].astype(str)).map(fq_encode)
        
        train[new_column] /= train[period+'_total']
        test[new_column]  /= test[period+'_total']
 
"""

'''0902
#20180825 ADD
for col in ['local_hour']:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
            
    train[col+'_total'] = train[col].map(fq_encode)
    test[col+'_total']  = test[col].map(fq_encode)

periods = ['local_hour']
i_cols = ['uid']
for period in periods:
    for col in i_cols:
        new_column = col + '_' + period
            
        temp_df = pd.concat([train[[col,period]], test[[col,period]]])
        temp_df[new_column] = temp_df[col].astype(str) + '_' + (temp_df[period]).astype(str)
        fq_encode = temp_df[new_column].value_counts().to_dict()
            
        train[new_column] = (train[col].astype(str) + '_' + train[period].astype(str)).map(fq_encode)
        test[new_column]  = (test[col].astype(str) + '_' + test[period].astype(str)).map(fq_encode)
        
        train[new_column] /= train[period+'_total']
        test[new_column]  /= test[period+'_total']
'''


In [0]:
"""
#https://www.kaggle.com/kyakovlev/ieee-ground-baseline-make-amount-useful-again
########################### ProductCD and M4 Target mean
for col in ['ProductCD','M4']:
    temp_dict = train.groupby([col])['isFraud'].agg(['mean']).reset_index().rename(
                                                        columns={'mean': col+'_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col+'_target_mean'].to_dict()

    train[col+'_target_mean'] = train[col].map(temp_dict)
    test[col+'_target_mean']  = test[col].map(temp_dict)
"""

In [0]:

'''0902
########################### Client Virtual ID
# Let's add some kind of client uID based on cardID and addr columns
# The value will be very specific for each client so we need to remove it
# from final features. But we can use it for aggregations.
train['uid'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)
test['uid'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)

train['uid2'] = train['uid'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card5'].astype(str)
test['uid2'] = test['uid'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card5'].astype(str)

train['uid3'] = train['uid2'].astype(str)+'_'+train['addr1'].astype(str)+'_'+train['addr2'].astype(str)
test['uid3'] = test['uid2'].astype(str)+'_'+test['addr1'].astype(str)+'_'+test['addr2'].astype(str)

train['uid4'] = train['uid3'].astype(str)+'_'+train['P_emaildomain'].astype(str)
test['uid4'] = test['uid3'].astype(str)+'_'+test['P_emaildomain'].astype(str)

train['uid5'] = train['uid3'].astype(str)+'_'+train['R_emaildomain'].astype(str)
test['uid5'] = test['uid3'].astype(str)+'_'+test['R_emaildomain'].astype(str)

# Add values remove list
new_columns = ['uid','uid2','uid3','uid4','uid5']
remove_features += new_columns

print('#'*10)
print('Most common uIds:')
for col in new_columns:
    print('#'*10, col)
    print(train[col].value_counts()[:10])

# Do Global frequency encoding 
i_cols = ['card1','card2','card3','card5'] + new_columns
train, test = frequency_encoding(train, test, i_cols, self_encoding=False)
'''
new_columns = ['uid','uid2','uid3','uid4','uid5']
remove_features += new_columns

In [0]:
'''0902
########################### TransactionAmt

#20190901 START
i_cols = ['TransactionAmt']
periods = ['DT_D']

temp_df = pd.concat([train[['TransactionDT']+i_cols+periods], test[['TransactionDT']+i_cols+periods]])
for period in periods:
    for col in i_cols:
        for df in [temp_df]:
            df.set_index(period)[col].plot(style='.', title=col, figsize=(15, 3))
            plt.show()

# Clip Values
train['TransactionAmt'] = train['TransactionAmt'].clip(0,5000)
test['TransactionAmt']  = test['TransactionAmt'].clip(0,5000)


train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)

# For our model current TransactionAmt is a noise
# https://www.kaggle.com/kyakovlev/ieee-check-noise
# (even if features importances are telling contrariwise)
# There are many unique values and model doesn't generalize well
# Lets do some aggregations
i_cols = ['TransactionAmt']
uids = ['card1','card2','card3','card5','uid','uid2','uid3','uid4','uid5','bank_type']
aggregations = ['mean','std']

# uIDs aggregations
train, test = uid_aggregation(train, test, i_cols, uids, aggregations)
 
# TransactionAmt Normalization
periods = ['DT_D','DT_W','DT_M']
for df in [train, test]:
    df = values_normalization(df, periods, i_cols)

#20190901 END
    
# Let's add some kind of client uID based on cardID ad addr columns
# The value will be very specific for each client so we need to remove it
# from final feature. But we can use it for aggregations.
#train['uid'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card4'].astype(str)
#test['uid'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card4'].astype(str)

#train['uid2'] = train['uid'].astype(str)+'_'+train['addr1'].astype(str)+'_'+train['addr2'].astype(str)
#test['uid2'] = test['uid'].astype(str)+'_'+test['addr1'].astype(str)+'_'+test['addr2'].astype(str)


# Check if Transaction Amount is common or not (we can use freq encoding here)
# In our dialog with model we are telling to trust or not to these values  
valid_card = train['TransactionAmt'].value_counts()
valid_card = valid_card[valid_card>10]
valid_card = list(valid_card.index)
    


# For our model current TransactionAmt is a noise (even when features importances are telling contrariwise)
# There are many unique values and model doesn't generalize well
# Lets do some aggregations
i_cols = ['card1','card2','card3','card5','uid','uid2']

for col in i_cols:
    for agg_type in ['mean', 'std']:
        new_col_name = col+'_TransactionAmt_'+agg_type
        temp_df = pd.concat([train[[col, 'TransactionAmt']], test[[col,'TransactionAmt']]])
        temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                columns={agg_type: new_col_name})
        
        temp_df.index = list(temp_df[col])
        temp_df = temp_df[new_col_name].to_dict()   
    
        train[new_col_name] = train[col].map(temp_df)
        test[new_col_name]  = test[col].map(temp_df)

# Small "hack" to transform distribution 
# (doesn't affect auc much, but I like it more)
# please see how distribution transformation can boost your score 
# (not our case but related)
# https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html
train['TransactionAmt'] = np.log1p(train['TransactionAmt'])
test['TransactionAmt'] = np.log1p(test['TransactionAmt'])
'''

In [0]:
'''0902
#20190901 
########################### D Columns
# From columns description we know that
# D1-D15: timedelta, such as days between previous transaction, etc.
# 1. I can't imagine normal negative timedelta values (Let's clip Values)
# 2. Normalize (Min-Max, Standard score) All D columns, except D1,D2,D9
# 3. Do some aggregations based on uIDs
# 4. Freaquency encoding
# 5. D1,D2 are clipped by max train_df values (let's scale it)
i_cols = ['D'+str(i) for i in range(1,16)]
uids = ['uid','uid2','uid3','uid4','uid5','bank_type']
aggregations = ['mean','std']

####### uIDs aggregations
train, test = uid_aggregation(train, test, i_cols, uids, aggregations)

####### Cleaning Neagtive values and columns transformations
for df in [train, test]:

    for col in i_cols:
        df[col] = df[col].clip(0) 
    
    # Lets transform D8 and D9 column
    # As we almost sure it has connection with hours
    df['D9_not_na'] = np.where(df['D9'].isna(),0,1)
    df['D8_not_same_day'] = np.where(df['D8']>=1,1,0)
    df['D8_D9_decimal_dist'] = df['D8'].fillna(0)-df['D8'].fillna(0).astype(int)
    df['D8_D9_decimal_dist'] = ((df['D8_D9_decimal_dist']-df['D9'])**2)**0.5
    df['D8'] = df['D8'].fillna(-1).astype(int)

####### Values Normalization
i_cols.remove('D1')
i_cols.remove('D2')
i_cols.remove('D9')
periods = ['DT_D','DT_W','DT_M']
for df in [train, test]:
    df = values_normalization(df, periods, i_cols)

for col in ['D1','D2']:
    for df in [train, test]:
        df[col+'_scaled'] = df[col]/train[col].max()
        
####### Global Self frequency encoding
# self_encoding=True because 
# we don't need original values anymore
i_cols = ['D'+str(i) for i in range(1,16)]
train, test = frequency_encoding(train, test, i_cols, self_encoding=True)
'''

In [0]:
#20190901
'''
########################### card3/card5 most common hour 
# card3 or card5 is a bank country?
# can we find:
# - the most popular Transaction Hour
# - the most popular Week Day
# and then find distance from it

# Prepare bank type feature
for df in [train, test]:
    df['bank_type'] = df['card3'].astype(str) +'_'+ df['card5'].astype(str)
remove_features.append('bank_type') 

encoding_mean = {
    1: ['DT_D','DT_hour','_hour_dist','DT_hour_mean'],
    2: ['DT_W','DT_day_week','_week_day_dist','DT_day_week_mean'],
    3: ['DT_M','DT_day_month','_month_day_dist','DT_day_month_mean'],
    }

encoding_best = {
    1: ['DT_D','DT_hour','_hour_dist_best','DT_hour_best'],
    2: ['DT_W','DT_day_week','_week_day_dist_best','DT_day_week_best'],
    3: ['DT_M','DT_day_month','_month_day_dist_best','DT_day_month_best'],   
    }

# Some ugly code here (even worse than in other parts)
for col in ['card3','card5','bank_type']:
    for df in [train, test]:
        for encode in encoding_mean:
            encode = encoding_mean[encode].copy()
            new_col = col + '_' + encode[0] + encode[2]
            df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)

            temp_dict = df.groupby([new_col])[encode[1]].agg(['mean']).reset_index().rename(
                                                                    columns={'mean': encode[3]})
            temp_dict.index = temp_dict[new_col].values
            temp_dict = temp_dict[encode[3]].to_dict()
            df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)

        for encode in encoding_best:
            encode = encoding_best[encode].copy()
            new_col = col + '_' + encode[0] + encode[2]
            df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)
            temp_dict = df.groupby([col,encode[0],encode[1]])[encode[1]].agg(['count']).reset_index().rename(
                                                                    columns={'count': encode[3]})

            temp_dict.sort_values(by=[col,encode[0],encode[3]], inplace=True)
            temp_dict = temp_dict.drop_duplicates(subset=[col,encode[0]], keep='last')
            temp_dict[new_col] = temp_dict[col].astype(str) +'_'+ temp_dict[encode[0]].astype(str)
            temp_dict.index = temp_dict[new_col].values
            temp_dict = temp_dict[encode[1]].to_dict()
            df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)
'''

In [0]:
"""
########################### Anomaly Search in geo information

# Let's look on bank addres and client addres matching
# card3/card5 bank country and name?
# Addr2 -> Clients geo position (country)
# Most common entries -> normal transactions
# Less common etries -> some anonaly
train['bank_type'] = train['card3'].astype(str)+'_'+train['card5'].astype(str)
test['bank_type']  = test['card3'].astype(str)+'_'+test['card5'].astype(str)

train['address_match'] = train['bank_type'].astype(str)+'_'+train['addr2'].astype(str)
test['address_match']  = test['bank_type'].astype(str)+'_'+test['addr2'].astype(str)

for col in ['address_match','bank_type']:
    temp_df = pd.concat([train[[col]], test[[col]]])
    temp_df[col] = np.where(temp_df[col].str.contains('nan'), np.nan, temp_df[col])
    temp_df = temp_df.dropna()
    fq_encode = temp_df[col].value_counts().to_dict()   
    train[col] = train[col].map(fq_encode)
    test[col]  = test[col].map(fq_encode)

train['address_match'] = train['address_match']/train['bank_type'] 
test['address_match']  = test['address_match']/test['bank_type']
"""

In [0]:
"""
train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')
"""

## ADD END

In [0]:
"""
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo','P_emaildomain','R_emaildomain']

useful_features.extend(['device_name', 'device_version', 'OS_id_30','version_id_30', 'browser_id_31', 'version_id_31','screen_width','screen_height', 'had_id',
                        'Missing_count','card_dt','P_emaildomain_1',	'P_emaildomain_2'	,'P_emaildomain_3',	'R_emaildomain_1'	,'R_emaildomain_2',	
                        'R_emaildomain_3','addr1_dt','mail_dt'])
"""


In [0]:
#cols_to_drop = [col for col in train.columns if col not in useful_features]
#cols_to_drop.remove('isFraud')
#cols_to_drop.remove('TransactionID')
#cols_to_drop.remove('TransactionDT')

In [0]:
#print('{} features are going to be dropped for being useless'.format(len(cols_to_drop)))

#train = train.drop(cols_to_drop, axis=1)
#test = test.drop(cols_to_drop, axis=1)

In [0]:
"""
# New feature - decimal part of the transaction amount
train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)

# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))

# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)
train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train[feature] = train[f1].astype(str) + '_' + train[f2].astype(str)
    test[feature] = test[f1].astype(str) + '_' + test[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values) + list(test[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
    test[feature] = le.transform(list(test[feature].astype(str).values))
    
for feature in ['id_34', 'id_36']:
    if feature in useful_features:
        # Count encoded for both train and test
        train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
        test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
        
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
    if feature in useful_features:
        # Count encoded separately for train and test
        train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
        test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))
"""

In [0]:
'''未使用
#https://www.kaggle.com/amirhmi/a-comprehensive-guide-to-get-0-9492
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other',
          'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft',
          'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 
          'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other',
          'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo',
          'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo',
          'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo',
          'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo',
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other',
          'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple',
          'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other',
          'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']
for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    

#train_missing_values = train.isnull().sum().sort_values(ascending=False) / len(train)
#test_missing_values = test.isnull().sum().sort_values(ascending=False) / len(test)
#train_missing_values = [str(x) for x in train_missing_values[train_missing_values > 0.80].keys()]
#test_missing_values = [str(x) for x in test_missing_values[test_missing_values > 0.80].keys()]

#dropped_columns = train_missing_values + test_missing_values
#print(dropped_columns)
'''


In [0]:
'''
########################### Encode Str columns
# For all such columns (probably not)
# we already did frequency encoding (numeric feature)
# so we will use astype('category') here
for col in list(train):
    if train[col].dtype=='O':
        print(col)
        train[col] = train[col].fillna('unseen_before_label')
        test[col]  = test[col].fillna('unseen_before_label')
        
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        
        le = LabelEncoder()
        le.fit(list(train[col])+list(test[col]))
        train[col] = le.transform(train[col])
        test[col]  = le.transform(test[col])
        
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')
'''

for col in tqdm_notebook(train.columns):
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [0]:
print(train.shape,test.shape)

##  pseudo_labeler

In [0]:
"""
#予測精度の高いテストデータを訓練データに加える
gdrive_dl("stack_gmean9498.csv")

lbl_df = pd.read_csv("stack_gmean9498.csv")
lbl_df = pd.merge(test, lbl_df.query('isFraud >= 0.95 | isFraud <= 0.05'),  on='TransactionID',how = 'inner')
lbl_df.loc[lbl_df['isFraud']  >= 0.95, 'isFraud'] = 1
lbl_df.loc[lbl_df['isFraud']  <= 0.05, 'isFraud'] = 0
print(train.shape,lbl_df.shape)
train = pd.concat([train, lbl_df])
print(train.shape)
"""

#### Negative Down Sampling

In [0]:
"""下がる
# Negative downsampling
train_pos = train[train['isFraud']==1]
train_neg = train[train['isFraud']==0]

train_neg = train_neg.sample(int(train.shape[0] * 0.2), random_state=42)
train = pd.concat([train_pos,train_neg]).sort_index()
"""

In [0]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
del train
gc.collect()

In [0]:
"""
plt.figure(figsize=(100, 100))
corr_mat = train.corr(method='pearson')
import seaborn as sns
sns.heatmap(corr_mat,
            vmin=-1.0,
            vmax=1.0,
            center=0,
            annot=True, # True:格子の中に値を表示
            fmt='.1f',
            xticklabels=corr_mat.columns.values,
            yticklabels=corr_mat.columns.values
           )
plt.show()
"""

In [0]:
print(X.shape,y.shape)

#### 0826 SMOTE

In [0]:
"""
#これでは雑なのでカラムに合わせて埋めたい
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(-999)
test = test.replace([np.inf, -np.inf], np.nan)
test = test.fillna(-999)
"""

In [0]:
#pip install imblearn

In [0]:
'''
from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y==0)))

sm = SMOTE(random_state=99, sampling_strategy = 0.2)
X_train_new, y_train_new = sm.fit_sample(X, y)

X_train_new = pd.DataFrame(X_train_new)
X_train_new.columns = X.columns
X = X_train_new.copy()

y_train_new = pd.DataFrame(y_train_new)
#y_train_new.columns = y_train.columns
y = y_train_new.copy()


print('After OverSampling, the shape of X_train_new: {}'.format(X_train_new.shape))
print('After OverSampling, the shape of y_train_new: {} \n'.format(y_train_new.shape))

del X_train_new,y_train_new
'''

## Feature Selection

In [0]:
#特徴選択
drop_features = ['R_mail_dt','R_emaildomain','R_emaildomain_1',	'R_emaildomain_2',	'R_emaildomain_3','Missing_count_diff',
                 'Missing_count_diffAbs','M1','uid','uid2','bank_type','V107',"card6_dt",'M_type','DeviceInfo','is_december','is_holiday',
                 'DT_M_total','DT_W_total','DT_D_total','C6','C8','C10','C11','C12','had_id','V27','V28','V79','V88','V117','V119','V120','V240','V305','email_check_nan_any','email_check_nan_all']#'DT_M_ABS',
drop_features += remove_features
"""
#https://www.kaggle.com/duykhanh99/hust-lgb-fe-0-9485-lb-newfeature
drop_features += ['V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
"""
#drop_features +=["M8_dt","M9_dt"]

In [0]:
X.info()

## Group K Fold

In [0]:
"本命"
params = {'num_leaves': 450,# 491
          'min_child_weight': 0.03454472573214212,#,0.05
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,#120,#106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006,#0.006883242363721497,0.01,0.05883242363721497
          "boosting_type":"gbdt",#"goss", #"dart",#"gbdt"
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 45,
         }
"""
#検証用
params = {'bagging_fraction': 0.9505876804775923,
 'bagging_freq': 5,
 'boost_from_average': 'true',
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8330328181873989,
 'is_unbalance': False,
 'learning_rate': 0.08,
 'max_depth': -1,
 'metric': 'auc',
 'min_data_in_leaf': 93,
 'min_sum_hessian_in_leaf': 0.8630063531750576,
 'nthread': -1,
 'num_boost_round': 92,
 'num_leaves': 330,
 'objective': 'binary',
 'reg_alpha': 1.7145713032508187,
 'reg_lambda': 1.6852918917932098,
 'seed': 24,
 'subsample_for_bin': 3847,
 'tree_learner': 'serial',
 'verbose': -1
  }
import xgboost as xgb
xgbclf = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=12,
    learning_rate=0.048,
    subsample=0.85,
    colsample_bytree=0.85,
    missing=np.nan,
    tree_method='hist',
    reg_alpha=0.15,
    reg_lamdba=0.85)
  """

In [0]:
from sklearn.model_selection import train_test_split, KFold, GroupKFold
folds = GroupKFold(n_splits=3)
split_groups = X['DT_M']

aucs = list()

print(X.shape,test.shape)
X = X.drop(drop_features, axis=1)
test = test.drop(drop_features, axis=1)
print(X.shape,test.shape)

feature_importances = pd.DataFrame()
feature_importances['feature'] = X.columns

training_start_time = time()

for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y, groups=split_groups)):
  start_time = time()
  print('Training on fold {}'.format(fold + 1))
    
  trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
  val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
  clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
  
  feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
  aucs.append(clf.best_score['valid_1']['auc'])
    
  print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
print('-' * 30)
print('Training has finished.')
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
print('Mean AUC:', np.mean(aucs))
print('-' * 30)

#### XGBM  START

In [0]:

"""
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit,KFold
n_fold = 5
folds = KFold(n_splits=n_fold,shuffle=True)
xgb_submission = sub
xgb_submission['isFraud'] = 0
from sklearn.metrics import roc_auc_score
for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
    xgbclf = xgb.XGBClassifier(
        n_estimators=1000,
        max_depth=12,
        learning_rate=0.048,
        subsample=0.85,
        colsample_bytree=0.85,
        missing=np.nan,
        tree_method='gpu_hist',
        reg_alpha=0.15,
        reg_lamdba=0.85
    )
    X = X.drop(drop_features, axis=1)
    X_train_, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train_, y_valid = y.iloc[train_index], y.iloc[valid_index]
    xgbclf.fit(X_train_,y_train_)
    del X_train_,y_train_
    pred=xgbclf.predict_proba(test)[:,1]
    val=xgbclf.predict_proba(X_valid)[:,1]
    del xgbclf, X_valid
    print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
    del val,y_valid
    xgb_submission['isFraud'] = xgb_submission['isFraud']+pred/n_fold
    del pred
    gc.collect()
"""

In [0]:
#xgb_submission.to_csv('xgb_' + '201900916' +'submission.csv', index=False)
#gdrive_up('xgb_'+ '20190825' +'submission.csv')
#!kaggle competitions submit -c ieee-fraud-detection -f ('xgb_'+ '20190825' +'submission.csv' -m '2019/8/21 XGB'

### XGBM END

In [0]:
#X = X.fillna(-999)
#describe_add_isnull(X)

### TimeSeriesSplit

In [0]:
"""
folds = TimeSeriesSplit(n_splits=5)

X = X.drop(drop_features, axis=1)
test = test.drop(drop_features, axis=1)

aucs = list()
feature_importances = pd.DataFrame()
feature_importances['feature'] = X.columns

training_start_time = time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    aucs.append(clf.best_score['valid_1']['auc'])
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
print('-' * 30)
print('Training has finished.')
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
print('Mean AUC:', np.mean(aucs))
print('-' * 30)
"""

In [0]:
feature_importances['average'] = feature_importances[['fold_{}'.format(fold + 1) for fold in range(folds.n_splits)]].mean(axis=1)
feature_importances.to_csv('feature_importances.csv')

plt.figure(figsize=(35, 35))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(200), x='average', y='feature');
plt.title('50 TOP feature importance over {} folds average'.format(folds.n_splits));

In [0]:
# clf right now is the last model, trained with 80% of data and validated with 20%
best_iter = clf.best_iteration

In [0]:

clf = lgb.LGBMClassifier(**params, num_boost_round=best_iter)
clf.fit(X, y)

In [0]:
print(X.shape,test.shape)

In [0]:
sub['isFraud'] = clf.predict_proba(test)[:, 1]
filename = params["boosting_type"] + "-" + str(np.mean(aucs)) + '-ieee_cis_fraud.csv'
sub.to_csv(filename, index=False)
gdrive_up(filename)

In [0]:
#!kaggle competitions submit -c ieee-fraud-detection -f gbdt-0.9400895987182839-ieee_cis_fraud.csv -m 921

In [0]:
!kaggle competitions submissions -c ieee-fraud-detection

fileName     date                 description  status    publicScore  privateScore  
-----------  -------------------  -----------  --------  -----------  ------------  
FIX1003.csv  2019-10-03 08:26:23  FIX1003.csv  complete  0.9535       None          
FIX1003.csv  2019-10-03 08:24:48  FIX1003.csv  complete  0.9535       None          
FIX1002.csv  2019-10-03 06:42:22  FIX1002.csv  complete  0.9535       None          
FIX1002.csv  2019-10-03 06:40:58  FIX1002.csv  complete  0.9535       None          
FIX1002.csv  2019-10-03 06:39:18  FIX1002.csv  complete  0.9535       None          
FIX1002.csv  2019-10-02 12:06:16  FIX1002.csv  complete  0.9535       None          
FIX1002.csv  2019-10-02 12:01:49  FIX1002.csv  complete  0.9534       None          
FIX1002.csv  2019-10-02 11:44:18  FIX1002.csv  complete  0.9533       None          
FIX1002.csv  2019-10-02 11:40:28  FIX1002.csv  complete  0.9534       None          
FIX1002.csv  2019-10-02 11:36:41  FIX1002.csv  complete  0.9534  

In [0]:
send_line_notification("終わったよ")

NameError: ignored